In [ ]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

**<h4>1. COMPARE PARAMETER VALUES</h4>**

**Choose desired parameter values and variables to plot below.**

In [ ]:
# Parameter values
values = [0.7, 2.67]

# Variables to plot PLEASE CHOOSE 12 VARIABLES
varl = ['P_C','P_Y','P_Y_KL','Y','C','C_Y','K','ell','E','r_K','r_ell','r_E',]

**Choose desired parameter, specify a chock, and run the model (might take some time).**

In [ ]:
s1={} # dict for sol
s2={} # dict for ss

for i,value in enumerate(values):
    
    from BabyMAKROModel import BabyMAKROModelClass
    
    model = BabyMAKROModelClass(name='baseline')
    
    par = model.par
    ss = model.ss
    sol = model.sol
    
    # choose parameter
    par.sigma_C_G = value
    
    # run model in SS
    model.find_ss(0.50)
    model.set_exo_ss()
    model.set_unknowns_ss()
    model.evaluate_blocks()
    errors = model.get_errors(do_print=True)
    
    # apply shock
    model.calc_jac()
    model.set_exo_ss()
    Tshock = 20
    shock = 0.2*ss.r_E
    persistence = 0.75**np.arange(Tshock)
    sol.r_E[:Tshock] = ss.r_E + shock*persistence

    # run model with shock
    model.set_unknowns_ss()
    model.evaluate_blocks()
    errors = model.get_errors()
    model.find_IRF()

    # save variables in dictonaries
    for var in varl:
        key = str(var+str(i)) 
        s1[key] = sol.__dict__[var]
        s2[key] = ss.__dict__[var]

**Plot the variables and see the effects of different parameter values.**

In [ ]:
T_IRF = 40

fig = plt.figure(figsize=(20,20/1.1))
first_leg = mpatches.Patch(color='#1f77b4', label='Parameter value:'+str(values[0]))
second_leg = mpatches.Patch(color='#ff7f0e', label='Parameter value:'+str(values[1]))
thrid_leg = mpatches.Patch(color='#2ca02c', label='Parameter value:'+str(values[2]))


for i,var in enumerate(varl):
    
    ax = fig.add_subplot(4,3,i+1)
    
    ax.set_ylabel('% of s.s.')
    ax.set_title(var)
     
    for j,value in enumerate(values):
        ax.plot((s1[str(var+str(j))][:T_IRF]/s2[str(var+str(j))]-1)*100)
    ax.legend(handles=[first_leg ,second_leg ,thrid_leg])


**<h4>2. COMPARE SHOCKS</h4>**

**Specify shocks and choose variables to plot below.**

In [ ]:
# specify shocks
var_l = ['P_E','r_E'] # variables to shock
Tshock_l = [20,20, 20] # length of shock (periods)
shock_l = [0.01,0.01, 0.01] # shock as pct. of ss
pers_l = [0.75,0.75,0.75] # persistence of shock

# Variables to plot PLEASE CHOOSE 12 VARIABLES
varl = ['P_C','P_Y','P_Y_KL','Y','C','C_Y','K','ell','E','r_K','r_ell','r_E']

**Run model for each shock.**

In [ ]:
s1={} # dict for sol
s2={} # dict for ss

for i,(var, Tshock, shock, pers) in enumerate(zip(var_l, Tshock_l, shock_l, pers_l)):
    
    from BabyMAKROModel import BabyMAKROModelClass
    
    model = BabyMAKROModelClass(name='baseline')
    
    par = model.par
    ss = model.ss
    sol = model.sol
    
    # run model in SS
    model.find_ss(0.50)
    model.set_exo_ss()
    model.set_unknowns_ss()
    model.evaluate_blocks()
    errors = model.get_errors(do_print=True)
    
    # apply shock
    model.calc_jac()
    model.set_exo_ss()
    shock = shock*ss.__dict__[var]
    pers = pers**np.arange(Tshock)
    sol.__dict__[var][:Tshock] = ss.__dict__[var] + shock*pers

    # run model with shock
    model.set_unknowns_ss()
    model.evaluate_blocks()
    errors = model.get_errors()
    model.find_IRF()

    # save variables in dictonaries
    for var in varl:
        key = str(var+str(i)) 
        s1[key] = sol.__dict__[var]
        s2[key] = ss.__dict__[var]

**Plot the variables and compare the shocks.**

In [ ]:
import matplotlib.patches as mpatches

T_IRF = 40

fig = plt.figure(figsize=(20,20/1.1))
first_leg = mpatches.Patch(color='#1f77b4', label='1 pct. shock to: '+str(var_l[0]))
second_leg = mpatches.Patch(color='#ff7f0e', label='1 pct. shock to: '+str(var_l[1]))
# third_leg = mpatches.Patch(color='#2ca02c', label='1 pct. shock to: '+str(var_l[2]))

for i,var in enumerate(varl):
    
    ax = fig.add_subplot(4,3,i+1)
    
    ax.set_ylabel('% of s.s.')
    ax.set_title(var)
     
    for j,value in enumerate(var_l):
        ax.plot((s1[str(var+str(j))][:T_IRF]/s2[str(var+str(j))]-1)*100)
    ax.legend(handles=[first_leg ,second_leg])


In [ ]:
sol.C/sol.Y

In [ ]:
sol.Y/sol.C_Y